## 収入調査データセット

データの取得
- https://cran.r-project.org/web/packages/arules/readme/README.html

データの説明
- https://www.rdocumentation.org/packages/arules/versions/1.6-6/topics/Income
- ショッピングモールでの収入とその他属性のアンケート結果

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint

income_columns = ['income', 'sex', 'marital status', 'age', 'education', 'occupation',
                  'years in bay area', 'dual incomes', 'number in household',
                  'number of children', 'householder status', 'type of home', 
                  'ethnic classification', 'language in home']

income = pd.read_csv('data/Income.csv', names=income_columns)
# 列ごとの欠損値の数
print(income.isna().sum())
# データの確認
income

### データの可視化

- 収入
- 収入と年齢の関係
- 収入と教育の関係

収入を表す income 列の頻度を表す棒グラフを描画

In [ ]:
# 収入 income の棒グラフ
sns.countplot(x=income.income)
plt.show()

クロス集計でデータの関係を表にまとめる

In [ ]:
# 収入と年齢の関係（クロス集計）
pd.crosstab(income.income, income.age)

In [ ]:
# 収入と教育の関係（クロス集計）
pd.crosstab(income.income, income.education)

年齢が高い、学歴が高い方が、収入が多い傾向は読み取れる。

### データ形式の変換

In [ ]:
# mlxtend が処理できる形態に変換
# 1. TIDごとのリストを作成
trans_data = income.to_numpy().tolist()

# 確認（最初の2行）
pprint(trans_data[0:2])

# 2. 作成したリストから横持ち形式（行がTID）のデータを作成
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_data = te.fit(trans_data).transform(trans_data)
df = pd.DataFrame(te_data, columns=te.columns_)

# 確認
df

### 相関ルールの生成

In [ ]:
# 最小サポート: 15%
min_sup = 0.15
# 最小コンフィデンス: 50%
min_confidence = 0.5

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

# Apriori アルゴリズムにより頻出アイテムセットを作成
frequent_itemsets = apriori(df, min_support=min_sup, use_colnames=True)

# 相関ルールの生成
# - 最小コンフィデンスを指定する場合
#   - metric='confidence' 
#   - min_threshold: 最小コンフィデンスの値
_df_rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=min_confidence)
df_rules = _df_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

# 確認
# - リフト値でソート（降順）
df_rules.sort_values('lift', ascending=False)

### 高収入の関連要因

相関ルールから、ルール A⇒B のBに ```income=$40,000+``` が含まれる相関ルールを表示

In [ ]:
df_rules_income_high = df_rules[df_rules['consequents'].\
    map(lambda x: True if 'income=$40,000+' in x else False)]
# 確認
df_rules_income_high

In [ ]:
df_rules_income_high.sort_values('lift', ascending=False)